# Visualization part
  In this part we find the test accuracy on the best model found after Question 3 and we did the following
   - Found the test accuracy
   - Plotted an image grid of 10X3 showing true vs predicted label based on best model
   - Visualize the filters of 1st Convolution layer

In [ ]:
!pip install wget

In [ ]:
!pip install wandb -qqq
import wandb
wandb.login()

wandb: Currently logged in as: mak109 (use `wandb login --relogin` to force relogin)


True

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import random
import wget
import os
from zipfile import ZipFile
from PIL import Image
import yaml
from matplotlib import gridspec
plt.rcParams["figure.figsize"] = (20,10)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Sequential,regularizers,optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
random.seed(123)

In [ ]:
#Image download and extracting
import wget
import os
from zipfile import ZipFile
url='https://storage.googleapis.com/wandb_datasets/nature_12K.zip'
filename = os.path.basename(url)

if not os.path.exists(filename) and not os.path.exists("inaturalist_12K"):
  filename = wget.download(url)
  with ZipFile(filename, 'r') as zip:
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')
  os.remove(filename)

In [ ]:
image_size = (256,256)
num_layers = 5
num_classes = 10

In [ ]:
def CNN(config):
    model = Sequential([
        layers.Input((image_size[0],image_size[1],3)),
        layers.Rescaling(1./255)
        ])
    
    for l in range(num_layers):
        model.add(layers.Conv2D(filters=config["filters_list"][l],kernel_size=(config["kernel_sizes"][l][0],config["kernel_sizes"][l][1]),
                        activation=config["activation"],padding="same",kernel_regularizer=regularizers.l2(config["weight_decay"])))
        if config["batch_normalization"] == 'True':
            model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2,2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(config["dense_layer_size"],activation=config["activation"],kernel_regularizer=regularizers.l2(config["weight_decay"])))
    model.add(layers.Dropout(config["dropout"]))

    model.add(layers.Dense(num_classes,activation="softmax"))
    return model

In [ ]:
# best_test_run_id = "8ohaf75c"
test_run_id = "8ohaf75c"
api = wandb.Api()
run = api.run("dlstack/CS6910-ASSIGNMENT-2/"+test_run_id)
if os.path.exists('model-best.h5'):
    os.remove('model-best.h5')
model_weights = wandb.restore("model-best.h5",run_path="dlstack/CS6910-ASSIGNMENT-2/"+test_run_id)
config = run.config

model = CNN(config)
model.load_weights(model_weights.name)

model.compile(
    optimizer=optimizers.Adam(learning_rate=config["learning_rate"]),
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
    )
test_generator = ImageDataGenerator(dtype=tf.float32,validation_split=0.0,data_format='channels_last').flow_from_directory(
        'inaturalist_12K/val',
        target_size = image_size,
        batch_size = config['batch_size'],
        color_mode = 'rgb',
        class_mode = 'sparse',
        seed=123
        )
test_loss,test_acc = model.evaluate(test_generator)
print(f"Test Accuracy : {test_acc} Test loss : {test_loss}")

In [ ]:
class_names =list(test_generator.class_indices)
# images,labels = next(iter(test_generator))
# pred_labels = tf.argmax(model.predict(images),1)
# np.sum((labels.astype('int') == pred_labels)) #print number of correct labels
test_generator_ = test_generator

In [ ]:
max_num_correct=0
for images,labels in test_generator_:
    pred_labels = tf.argmax(model.predict(images),1)
    num_correct = np.sum((labels.astype('int')[:30] == pred_labels[:30]))
    print(num_correct)
    if num_correct > max_num_correct:
        max_num_correct = num_correct
        best_images = images
        best_labels = labels
        best_pred_labels = pred_labels
    if(max_num_correct >= 17):
        break

In [ ]:
images,labels,pred_labels = best_images,best_labels,best_pred_labels

In [ ]:
wandb.init(project="CS6910-ASSIGNMENT-2",entity="dlstack")
idx=0
fig = plt.figure(figsize=(60,60))
# fig.tight_layout()
g = gridspec.GridSpec(10,3,wspace=0.1,left=0.7,right=0.9)
for i in range(10):
    for j in range(3):
        ax = plt.subplot(g[i,j])
        ax.imshow(images[idx].astype("uint8"))
        title = f"True label : {class_names[labels[idx].astype('int')]}\nPredicted label : {class_names[pred_labels[idx]]}"
        ax.set_title(title)
        ax.axis("off")
        idx += 1
plt.savefig("test_image_37percent.jpg",bbox_inches="tight")
wandb.run.name= "image-test-37-1"
wandb.log({"test_image_accuracy_over_30_images":wandb.Image("test_image_37percent.jpg")})
wandb.finish()

In [ ]:
# biases_conv1 = model.layers[1].weights[1].numpy()
# weights_conv2 = model.layers[1].weights[0].numpy()

In [ ]:
wandb.init(project="CS6910-ASSIGNMENT-2",entity="dlstack")
test_generator_ = test_generator
images,labels = next(iter(test_generator_))
rand_indx = random.randint(0,32)
random_image = images[rand_indx].astype("uint8")
random_image_label = labels[rand_indx].astype("int")
filters = model.layers[1].call(images).numpy()[rand_indx]
plt.figure(figsize=(10,10))
plt.imshow(random_image)
plt.axis("off")
plt.title("Random image label : "+str(class_names[random_image_label]))
plt.savefig("random_image_best_model.jpg",bbox_inches="tight")

idx=0
fig = plt.figure(figsize=(60,60))
# fig.tight_layout()
g = gridspec.GridSpec(4,8,wspace=0.1,left=0.7,right=0.9,top=0.7,bottom=0.6)
for i in range(4):
    for j in range(8):
        ax = plt.subplot(g[i,j])
        ax.imshow(filters[:,:,idx].astype("uint8"))
        title = "filter : "+str(idx+1)
        ax.set_title(title)
        ax.axis("off")
        idx += 1
plt.savefig("random_image_best_model_filters.jpg",bbox_inches="tight")

wandb.run.name = "ran_img_best_model_filters"
wandb.log({"Random Image best model : ":wandb.Image("random_image_best_model.jpg")})
wandb.log({"Random Image best model filters: ":wandb.Image("random_image_best_model_filters.jpg")})
wandb.finish()

In [ ]:
# plt.imshow(model.layers[1].apply(images).numpy()[rand_indx][:,:,0].astype("uint8"))

In [ ]:
config